In [198]:
import pandas as pd
import numpy as np

In [241]:
# Question 1
referendum = pd.read_csv('data/referendum.csv', sep=';')
regions = pd.read_csv('data/regions.csv')
departments = pd.read_csv('data/departments.csv')

In [242]:
# Question 2
merged_df = pd.merge(departments, regions, right_on='code', left_on='region_code', how='left')
df =  merged_df.rename(
    columns={
        'region_code': 'code_reg', 
        'code_x': 'code_dep', 
        'name_x': 'name_dep', 
        'name_y':'name_reg'
        })
        
r_d = df[['code_reg', 'name_reg', 'code_dep', 'name_dep']]

In [243]:
# Question 3

def standardize_dep_code(code):
    if (code == '2A') | (code == '2B'):
        return code
    
    else: return '0' + str(int(code))

referendum = referendum.drop(referendum[referendum['Department code'].str.startswith('Z')].index)
referendum['Department code'] = referendum['Department code'].map(standardize_dep_code)

r_d = r_d.drop(r_d[r_d['code_reg'].isin(['DOM', '01', '02', '03', '04', '06'])].index)
r_d['code_dep'] = r_d['code_dep'].map(standardize_dep_code)

new_df = pd.merge(r_d, referendum, left_on='code_dep', right_on='Department code', how='right')
new_df = new_df[[
        'Department code', 'Department name', 'Town code', 'Town name',
        'Registered', 'Abstentions', 'Null', 'Choice A', 'Choice B',
        'code_dep', 'code_reg', 'name_reg', 'name_dep'
    ]]

In [272]:
q3_df = new_df.groupby(['code_reg', 'name_reg'], as_index=False).sum()
q3_df = q3_df.set_index(['code_reg']).drop(columns=['Town code'])
q3_df.head()

,name_reg,Registered,Abstentions,Null,Choice A,Choice B
code_reg,,,,,,
11,Île-de-France,6942361,1515884,85997,2536620,2803860
24,Centre-Val de Loire,1801366,326560,28827,590890,855089
27,Bourgogne-Franche-Comté,1997925,358321,33671,668246,937687
28,Normandie,2372779,426075,35513,816972,1094219
32,Hauts-de-France,4210444,866555,60601,1411257,1872031


In [273]:
q3_df = q3_df.set_index(['name_reg'])

print(q3_df['Registered'].sum() == 43_262_592)
print(q3_df.loc['Normandie', 'Abstentions'] == 426_075)
print(q3_df.loc['Grand Est', 'Choice A'] == 1_088_684)
print(q3_df.loc['Occitanie', 'Null'] == 62_732)

True
True
True
True


In [274]:
import pprint

In [276]:
geo_json = pd.read_json('region.geojson')
geo_json

ValueError: Expected object or value